In [1]:
import JSSP
import gym
import time
import numpy as np

# Environment Initialization

In [2]:
env_name = "JSSP-v0"
env = gym.make(env_name, instance_path = "instance3.txt")
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):


Observation space: Dict(0: Box(0, 9, (10,), int64), 1: Box(-2, 10, (10,), int64))
Action space: Box(-1, 10, (10,), int64)


/Users/boyanggu/RL-Research-2022-Summer/JSSP/envs/JSSP_env.py:119: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_ap

# Action Choosing

In [3]:
def get_legal_action(env):
    legal_actions = env.get_legal_actions()
    legal_action = []
    for job in range(len(legal_actions)):
        if len(legal_actions[job]) == 0:
            legal_action.append(-1)
        else:
            legal_job_actions = [machine for machine in legal_actions[job] if machine not in legal_action]
            legal_job_actions.append(-1)
            legal_action.append(np.random.choice(legal_job_actions))
    return np.array(legal_action)

In [4]:
def get_efficient_legal_action(env):
    legal_actions = env.get_legal_actions()
    legal_action = []
    for job in range(len(legal_actions)):
        if len(legal_actions[job]) == 0:
            legal_action.append(-1)
        else:
            legal_job_actions = [machine for machine in legal_actions[job] if machine not in legal_action]
            if len(legal_job_actions) == 0:
                legal_action.append(-1)
            else:
                legal_action.append(np.random.choice(legal_job_actions))
    return np.array(legal_action)


# Baselines

## 1. Random Sampling

In [6]:
state = env.reset()
episodes = 50
max_score = -100000
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = get_legal_action(env)
        # if not (np.all(action == -1)):
        #     print('Episode:{} Action:{} Time:{}'.format(episode, action, env.time))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Total_reward:{}'.format(episode, score))
    max_score = max(max_score, score)
print('From {} Episodes best policy has reward {}'.format(episodes, max_score))

Episode:1 Total_reward:-1264
Episode:2 Total_reward:-1324
Episode:3 Total_reward:-1196
Episode:4 Total_reward:-1242
Episode:5 Total_reward:-1371
Episode:6 Total_reward:-1349
Episode:7 Total_reward:-1245
Episode:8 Total_reward:-1206
Episode:9 Total_reward:-1182
Episode:10 Total_reward:-1198
Episode:11 Total_reward:-1232
Episode:12 Total_reward:-1255
Episode:13 Total_reward:-1200
Episode:14 Total_reward:-1164
Episode:15 Total_reward:-1211
Episode:16 Total_reward:-1225
Episode:17 Total_reward:-1138
Episode:18 Total_reward:-1076
Episode:19 Total_reward:-1216
Episode:20 Total_reward:-1204
Episode:21 Total_reward:-1190
Episode:22 Total_reward:-1199
Episode:23 Total_reward:-1218
Episode:24 Total_reward:-1265
Episode:25 Total_reward:-1268
Episode:26 Total_reward:-1195
Episode:27 Total_reward:-1277
Episode:28 Total_reward:-1226
Episode:29 Total_reward:-1217
Episode:30 Total_reward:-1191
Episode:31 Total_reward:-1272
Episode:32 Total_reward:-1261
Episode:33 Total_reward:-1191
Episode:34 Total_re

In [7]:
state = env.reset()
episodes = 50
max_score = -100000
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = get_efficient_legal_action(env)
        # if not (np.all(action == -1)):
        #     print('Episode:{} Action:{} Time:{}'.format(episode, action, env.time))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Total_reward:{}'.format(episode, score))
    max_score = max(max_score, score)
print('From {} Episodes best policy has reward {}'.format(episodes, max_score))

Episode:1 Total_reward:-1244
Episode:2 Total_reward:-1244
Episode:3 Total_reward:-1244
Episode:4 Total_reward:-1244
Episode:5 Total_reward:-1244
Episode:6 Total_reward:-1244
Episode:7 Total_reward:-1244
Episode:8 Total_reward:-1244
Episode:9 Total_reward:-1244
Episode:10 Total_reward:-1244
Episode:11 Total_reward:-1244
Episode:12 Total_reward:-1244
Episode:13 Total_reward:-1244
Episode:14 Total_reward:-1244
Episode:15 Total_reward:-1244
Episode:16 Total_reward:-1244
Episode:17 Total_reward:-1244
Episode:18 Total_reward:-1244
Episode:19 Total_reward:-1244
Episode:20 Total_reward:-1244
Episode:21 Total_reward:-1244
Episode:22 Total_reward:-1244
Episode:23 Total_reward:-1244
Episode:24 Total_reward:-1244
Episode:25 Total_reward:-1244
Episode:26 Total_reward:-1244
Episode:27 Total_reward:-1244
Episode:28 Total_reward:-1244
Episode:29 Total_reward:-1244
Episode:30 Total_reward:-1244
Episode:31 Total_reward:-1244
Episode:32 Total_reward:-1244
Episode:33 Total_reward:-1244
Episode:34 Total_re

## 2. Q-Learning

In [ ]:
# 1. Load Environment and Q-table structure
env_name = "JSSP-v0"
env = gym.make(env_name, instance_path = "instance1.txt")
Q = {}
# 2. Parameters of Q-learning
eta = .628
gma = .9
epis = 50
rev_list = [] # rewards per episode calculate
# 3. Q-learning Algorithm
for i in range(epis):
    # Reset environment
    s = env.reset()
    rAll = 0
    d = False
    # The Q-Table learning algorithm
    while not d:
        # env.render()
        # Choose action from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state & reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + eta*(r + gma*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
    rev_list.append(rAll)
    print("Episode: " + str(i) + " has reward " + str(rAll))
    # env.render()
# Code will stop at d == True, and render one state before it
print("Reward Sum on all episodes " + str(sum(rev_list)/epis))
print("Final Values Q-Table")
print(Q)

In [ ]:
state = env.reset()
episodes = 50
max_score = -100000
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = get_legal_action(env)
        # if not (np.all(action == -1)):
        #     print('Episode:{} Action:{} Time:{}'.format(episode, action, env.time))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Total_reward:{}'.format(episode, score))
    max_score = max(max_score, score)
print('From {} Episodes best policy has reward {}'.format(episodes, max_score))

## 2. Q-Learning

In [ ]:
# 1. Load Environment and Q-table structure
env_name = "JSSP-v0"
env = gym.make(env_name, instance_path = "instance1.txt")
Q = {}
# 2. Parameters of Q-learning
eta = .628
gma = .9
epis = 50
rev_list = [] # rewards per episode calculate
# 3. Q-learning Algorithm
for i in range(epis):
    # Reset environment
    s = env.reset()
    rAll = 0
    d = False
    # The Q-Table learning algorithm
    while not d:
        # env.render()
        # Choose action from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state & reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + eta*(r + gma*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
    rev_list.append(rAll)
    print("Episode: " + str(i) + " has reward " + str(rAll))
    # env.render()
# Code will stop at d == True, and render one state before it
print("Reward Sum on all episodes " + str(sum(rev_list)/epis))
print("Final Values Q-Table")
print(Q)

# Baselines

## 1. Random Sampling

In [ ]:
state = env.reset()
episodes = 50
max_score = -100000
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = get_legal_action(env)
        # if not (np.all(action == -1)):
        #     print('Episode:{} Action:{} Time:{}'.format(episode, action, env.time))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Total_reward:{}'.format(episode, score))
    max_score = max(max_score, score)
print('From {} Episodes best policy has reward {}'.format(episodes, max_score))

## 2. Q-Learning

In [ ]:
# 1. Load Environment and Q-table structure
env_name = "JSSP-v0"
env = gym.make(env_name, instance_path = "instance1.txt")
Q = {}
# 2. Parameters of Q-learning
eta = .628
gma = .9
epis = 50
rev_list = [] # rewards per episode calculate
# 3. Q-learning Algorithm
for i in range(epis):
    # Reset environment
    s = env.reset()
    rAll = 0
    d = False
    # The Q-Table learning algorithm
    while not d:
        # env.render()
        # Choose action from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state & reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + eta*(r + gma*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
    rev_list.append(rAll)
    print("Episode: " + str(i) + " has reward " + str(rAll))
    # env.render()
# Code will stop at d == True, and render one state before it
print("Reward Sum on all episodes " + str(sum(rev_list)/epis))
print("Final Values Q-Table")
print(Q)